In [280]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [281]:
#Table B - Cities Population Table 

url1 = "https://en.wikipedia.org/wiki/List_of_towns_and_cities_with_100,000_or_more_inhabitants/country:_A-B"
url2 = "https://en.wikipedia.org/wiki/List_of_towns_and_cities_with_100,000_or_more_inhabitants/country:_C-D-E-F"
url3 = "https://en.wikipedia.org/wiki/List_of_towns_and_cities_with_100,000_or_more_inhabitants/country:_G-H-I-J-K"
url4 = "https://en.wikipedia.org/wiki/List_of_towns_and_cities_with_100,000_or_more_inhabitants/country:_L-M-N-O"
url5 = "https://en.wikipedia.org/wiki/List_of_towns_and_cities_with_100,000_or_more_inhabitants/country:_P-Q-R-S"
url6 = "https://en.wikipedia.org/wiki/List_of_towns_and_cities_with_100,000_or_more_inhabitants/country:_T-U-V-W-Y-Z"

soup1 = BeautifulSoup(requests.get(url1).content, "html.parser")
soup2 = BeautifulSoup(requests.get(url2).content, "html.parser")
soup3 = BeautifulSoup(requests.get(url3).content, "html.parser")
soup4 = BeautifulSoup(requests.get(url4).content, "html.parser")
soup5 = BeautifulSoup(requests.get(url5).content, "html.parser")
soup6 = BeautifulSoup(requests.get(url6).content, "html.parser")

In [282]:
B_table_1 = soup1.find_all("table")[:-2]
#len(B_table_1) 25 tables
#print(len(soup1.find_all("h2")[1:-2])) 25 titles (country name)
B_table_1_countries = soup1.find_all("h2")[1:-2]
# print(B_table_1[-1], " ", B_table_1_countries[-1]) Aligned

In [283]:
B_table_2 = soup2.find_all("table")[:-2]
#print(len(B_table_2)) #25 tables
#print(len(soup2.find_all("h2")[1:-2])) #25 titles (country name)
B_table_2_countries = soup2.find_all("h2")[1:-2]
#print(B_table_2[-1], " ", B_table_2_countries[-1]) #Aligned

In [284]:
B_table_3 = soup3.find_all("table")[:-2]
#print(len(B_table_3)) #25 tables
#print(len(soup3.find_all("h2")[1:-2])) #25 titles (country name)
B_table_3_countries = soup3.find_all("h2")[1:-2]
#print(B_table_3[-1], " ", B_table_3_countries[-1]) #Aligned

In [285]:
B_table_4 = soup4.find_all("table")[:-2]
#print(len(B_table_4)) #25 tables
#print(len(soup4.find_all("h2")[1:-2])) #25 titles (country name)
B_table_4_countries = soup4.find_all("h2")[1:-2]
#print(B_table_4[-1], " ", B_table_4_countries[-1]) #Aligned

In [286]:
B_table_5 = soup5.find_all("table")[:-2]
#print(len(B_table_5)) #25 tables
#print(len(soup5.find_all("h2")[1:-2])) #25 titles (country name)
B_table_5_countries = soup5.find_all("h2")[1:-2]
#print(B_table_5[-1], " ", B_table_5_countries[-1]) #Aligned

In [287]:
B_table_6 = soup6.find_all("table")[:-2]
#print(len(B_table_6)) #25 tables
#print(len(soup6.find_all("h2")[1:-2])) #25 titles (country name)
B_table_6_countries = soup6.find_all("h2")[1:-2]
#print(B_table_6[-1], " ", B_table_6_countries[-1]) #Aligned

In [288]:
B_headers = ['City ID', 'City', 'Region', 'Country', 'Population', 'Population Year']

In [289]:
B_tables =[B_table_1, B_table_2, B_table_3, B_table_4, B_table_5, B_table_6]
B_tables_countries = [B_table_1_countries, B_table_2_countries, B_table_3_countries, B_table_4_countries, B_table_5_countries, B_table_6_countries]
#print(B_tables[5])

In [290]:
def remove_until_parenthesis(s):
    for i in range(len(s) - 1, -1, -1):
        if s[i] == ')':
            return s[:i+1]
    return s
def remove_all_but_number(m):
    v = ""
    for i in range(len(m)):
        if m[i].isdigit():
            v += m[i]  
    return v  

In [291]:
def get_pop_year(t):
    list = []
    length = len(t)
    for n in range(length):
        ro = t[n].find_all("th")
        #print(ro)
        raw =ro[-1].text.strip().replace(" ", "").split("(")
        if len(raw)>1:
            try:
                raw[1] = int(remove_until_parenthesis(raw[1]).replace(")", ""))
            except ValueError:
                raw[1] = int(remove_all_but_number(raw[1]))
        else: raw.insert(1, -1)
        list.append(raw[1])
    return list

In [292]:
TABLE_B = pd.DataFrame(columns=B_headers)
count = 0
for l in range(len(B_tables)):
    table = B_tables[l]
    country_list = B_tables_countries[l]
    pop_list = get_pop_year(table)    
    for i in range(len(table)):
        country = country_list[i].text.strip()
        pop_year = pop_list[i]
        B_row = table[i].find_all("tr")
        for row in B_row:
            row_data = row.find_all('td')
            indiv_row_data = [data.text.strip() for data in row_data]
            if indiv_row_data != []:
                if len(indiv_row_data) < 3:
                    indiv_row_data.insert(1, "") 
                indiv_row_data.insert(2, country)
                indiv_row_data.append(pop_year)
                try:
                    indiv_row_data[3] = int(indiv_row_data[3].replace(",", ""))
                except ValueError:
                    if indiv_row_data[0] == "Mogadishu":
                        pl = indiv_row_data[3].split("(")
                        indiv_row_data[3] = int(pl[0].replace(",", ""))
                        indiv_row_data[4] = int(remove_all_but_number(pl[1][:-3]))
                count +=1
                indiv_row_data.insert(0, count)
                for b in range(1,3):
                    if indiv_row_data[b].find(",") != -1:
                        indiv_row_data[b] = '"'+indiv_row_data[b] + '"'
                table_length = len(TABLE_B)
                TABLE_B.loc[table_length] = indiv_row_data
                print(indiv_row_data)


[1, 'Ghazni', 'Ghazni', 'Afghanistan', 190400, 2015]
[2, 'Herat', 'Herat', 'Afghanistan', 592900, 2015]
[3, 'Jalalabad', 'Nangarhar', 'Afghanistan', 280600, 2015]
[4, 'Kabul', 'Kabul', 'Afghanistan', 4601700, 2015]
[5, 'Kandahar', 'Kandahar', 'Afghanistan', 651400, 2015]
[6, 'Khost', 'Khost', 'Afghanistan', 160214, 2015]
[7, 'Kunduz', 'Kunduz', 'Afghanistan', 268800, 2015]
[8, 'Lashkargah', 'Helmand', 'Afghanistan', 201500, 2015]
[9, 'Mazar-i-Sharif', 'Balkh', 'Afghanistan', 500200, 2015]
[10, 'Mihtarlam', 'Laghman', 'Afghanistan', 144162, 2015]
[11, 'Puli Khumri', 'Baghlan', 'Afghanistan', 221200, 2015]
[12, 'Sar-e Pol', 'Sar-e Pol', 'Afghanistan', 164500, 2015]
[13, 'Sheberghan', 'Jowzjan', 'Afghanistan', 175599, 2015]
[14, 'Taloqan', 'Takhar', 'Afghanistan', 196400, 2015]
[15, 'Tirana', 'Tirana', 'Albania', 418495, 2011]
[16, 'Durrës', 'Durrës', 'Albania', 113249, 2011]
[17, 'Aflou', 'Laghouat', 'Algeria', 102000, 2008]
[18, 'Aïn Beïda', 'Oum El Bouaghi', 'Algeria', 116000, 2008]
[1

In [293]:
TABLE_B

,City ID,City,Region,Country,Population,Population Year
0,1,Ghazni,Ghazni,Afghanistan,190400,2015
1,2,Herat,Herat,Afghanistan,592900,2015
2,3,Jalalabad,Nangarhar,Afghanistan,280600,2015
3,4,Kabul,Kabul,Afghanistan,4601700,2015
4,5,Kandahar,Kandahar,Afghanistan,651400,2015
...,...,...,...,...,...,...
4625,4626,Mutare,Manicaland,Zimbabwe,224804,2022
4626,4627,Epworth,Harare,Zimbabwe,206368,2022
4627,4628,Gweru,Midlands,Zimbabwe,159050,2022
4628,4629,Kwekwe,Midlands,Zimbabwe,119863,2022


In [294]:
TABLE_B.to_csv("tableB.csv", index=False)